In [418]:
import os
import csv
import random
import operator
from copy import deepcopy
from pathlib import Path
from io import BytesIO

from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import LETTER
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER
from reportlab.lib.units import mm
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.graphics.shapes import Drawing, Line, Rect
from reportlab.lib import colors

from asfpy import asfpy

class ASFPFeedbackCanvas(Canvas):

    def __init__(self, *args, **kwargs):
        Canvas.__init__(self, *args, **kwargs)
        self._saved_page_states = []

    def showPage(self):
        self._saved_page_states.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        """add page info to each page (page x of y)"""
        num_pages = len(self._saved_page_states)
        for state in self._saved_page_states:
            self.__dict__.update(state)
            Canvas.showPage(self)
        Canvas.save(self)
        
class ASFPFeedbackPDF:


    def __init__(self, buffer, pagesize, logo, header_body):
        self.buffer = buffer
        if pagesize == 'A4':
            self.pagesize = A4
        elif pagesize == 'Letter':
            self.pagesize = LETTER
        self.width, self.height = self.pagesize
        
        self.logo = logo
        self.header_body = header_body
    
    def print_feedback(self, feedback_list):
        buffer = self.buffer
        doc = SimpleDocTemplate(buffer,
                                rightMargin=72,
                                leftMargin=72,
                                topMargin=72,
                                bottomMargin=72,
                                pagesize=self.pagesize)
 
        # Our container for 'Flowable' objects
        elements = []
 
        # A large collection of style sheets pre-made for us
        styles = getSampleStyleSheet()
        styles.add(ParagraphStyle(name='ASFPTitle', 
                                  fontName="Times-Roman", 
                                  fontSize=16,
                                  alignment=TA_CENTER))
        styles.add(ParagraphStyle(name='ASFPHeaderBody',
                                  fontName="Times-Roman",
                                  fontSize=11))
        styles.add(ParagraphStyle(name='ASFPEditorHeader',
                                  fontName="Times-Bold",
                                  fontSize=14,
                                  alignment=TA_CENTER))
        styles.add(ParagraphStyle(name='ASFPEditorFeedback',
                                  fontName="Times-Roman",
                                  fontSize=11))
        
        # Header creation
        logo_image = Image(self.logo, 1 * inch, 1 * inch)
        elements.append(logo_image)
        
        elements.append(Spacer(1, 0.25 * inch))
        elements.append(Paragraph('ASFP Editor Feedback Report', styles['ASFPTitle']))
        elements.append(Spacer(1, 0.25 * inch))
        
        elements.append(Paragraph("Dear " + feedback_list[0]["applicant_name"] + ",", styles['ASFPHeaderBody']))
        elements.append(Spacer(1, 12))
        elements.append(Paragraph(self.header_body, styles['ASFPHeaderBody']))
        elements.append(Spacer(1, 12))
        elements.append(Paragraph("- The ASFP Team", styles['ASFPHeaderBody']))
        
        D = Drawing(0, 0)
        D.add(Rect(0 * inch, 0, 6.25 * inch, 1, fillColor=colors.black))   
        
        for index, item in enumerate(feedback_list):
            
            editor_header = "EDITOR " + str(index + 1) + " (" + item["editor_role"] + ")"
            feedback = item["editor_feedback"].replace("\n\n\n", "\n").replace("\n\n", "\n").replace("\n", "<br/><br/>")
            
            elements.append(Spacer(1, 0.25 * inch))
            elements.append(D)
            elements.append(Spacer(1, 0.25 * inch))
            elements.append(Paragraph(editor_header, styles['ASFPEditorHeader']))
            elements.append(Spacer(1, 0.25 * inch))
            elements.append(Paragraph(feedback, styles["ASFPEditorFeedback"]))
 
        doc.build(elements,
                  canvasmaker=ASFPFeedbackCanvas)

In [419]:
FOLDER = "asfp2020"
LOGO_FILE = "logo.jpg"

path = Path("__file__").parent.absolute()

logo_filename = path.parent.joinpath(FOLDER).joinpath(LOGO_FILE)

buffer = BytesIO()

logo = logo_filename

header_body = "Thank you so much for participating in the inaugural Application Statement Feedback Program! You will find your feedback from two separate editors below. Editors used a variety of different approaches/rubrics, so your feedback will likely vary significantly. As a reminder, this feedback will not include any copy-editing, so please be sure to review your statements for any grammatical errors. These are just two opinions. It is always good to seek advice from many sources. We hope that these notes will be useful to you and wish you the best of luck with your graduate school applications!"

In [430]:
feedback_list = asfpy.read_list_csv(path.parent.joinpath(FOLDER).joinpath("test-feedback.csv"))

from itertools import groupby

newList = []
for k,v in groupby(feedback_list, key = lambda x:x["applicant_id"]):
    newList.append(list(v))

In [431]:
for feedback_item in newList:
    
    report = ASFPFeedbackPDF(buffer, 'Letter', logo, header_body)
    pdf = report.print_feedback(feedback_item)
    buffer.seek(0)
    
    output_filename = path.parent.joinpath(FOLDER).joinpath(feedback_item[0]["applicant_id"] + ".pdf")
    
    with open(output_filename, 'wb') as f:
        f.write(buffer.read())